In [1]:
import re
import pandas as pd
import polars as pl
import requests
from io import StringIO

In [17]:
word_forms = ["quiz", "quizzes"]

In [18]:
word_forms = [re.sub(r'([a-zA-Z0-9])-([a-zA-Z0-9])', r'\1 - \2', wf) for wf in word_forms]

In [19]:
word_forms = [wf.strip() for wf in word_forms]

In [20]:
word_forms

['quiz', 'quizzes']

In [21]:
n = [len(re.findall(r'\S+', wf)) for wf in word_forms]
n = list(set(n))

In [22]:
n

[1]

In [23]:
gram = [wf[:2] if n[0] > 1 else wf[:1] for wf in word_forms]

In [25]:
gram = list(set([g.lower() for g in gram]))

In [26]:
gram

['q']

In [47]:
repo = f"http://storage.googleapis.com/books/ngrams/books/googlebooks-eng-all-{n[0]}gram-20120701-{gram[0]}.gz"

In [48]:
repo

'http://storage.googleapis.com/books/ngrams/books/googlebooks-eng-all-1gram-20120701-q.gz'

In [49]:
word_forms = [re.sub(
    r'(\.|\?|\$|\^|\)|\(|\}|\{|\]|\[|\*)',
    r'\\\1', wf
    ) for wf in word_forms]
grep_words = "|".join([f"^{wf}$" for wf in word_forms])

In [63]:
test = 'r"(?i)' + grep_words + '"'

In [64]:
test

'r"(?i)^quiz$|^quizzes$"'

In [53]:
df = pl.scan_csv(repo, separator='\t', has_header=False)

In [54]:
df.schema

/var/folders/5y/_jt611y92qv80hh6jwzmjt940000gn/T/ipykernel_29223/3182147272.py:1: PerformanceWarning: Resolving the schema of a LazyFrame is a potentially expensive operation. Use `LazyFrame.collect_schema()` to get the schema without this warning.
  df.schema


Schema([('column_1', String),
        ('column_2', Int64),
        ('column_3', Int64),
        ('column_4', Int64)])

In [68]:
filtered_df = df.filter(pl.col("column_1").str.contains(r"(?i)" + grep_words))

In [69]:
result = filtered_df.collect()

In [70]:
result

column_1,column_2,column_3,column_4
str,i64,i64,i64
"""quizzes""",1515,1,1
"""quizzes""",1787,1,1
"""quizzes""",1793,1,1
"""quizzes""",1796,2,1
"""quizzes""",1799,2,2
…,…,…,…
"""QuiZ""",2004,18,4
"""QuiZ""",2005,7,3
"""QuiZ""",2006,46,4
